In [2]:
# Tratar de inputar los precios que faltan por expresiones regulares
# Eliminar todas filas que todo es nulo (check)
# Checar el tema de las superficies
# Precios: unificar todos los precios a dolares
# Hacer la multiplicacion de price por la superficie
# Imputar superficies nulas por medias condicionadas por barrio o cantidad de rooms

# Importacion de librerias utilizadas

In [3]:
import pandas as pd
import re

# Carga de la data de Properatti como un DataFrame de Pandas

In [4]:
data_location = "../Data/properatti.csv"

data = pd.read_csv(data_location)

data.head(3)

Unnamed: 0 operation property_type place_name  \
0           0      sell            PH  Mataderos   
1           1      sell     apartment   La Plata   
2           2      sell     apartment  Mataderos   

                       place_with_parent_names country_name  \
0        |Argentina|Capital Federal|Mataderos|    Argentina   
1  |Argentina|Bs.As. G.B.A. Zona Sur|La Plata|    Argentina   
2        |Argentina|Capital Federal|Mataderos|    Argentina   

               state_name  geonames_id                  lat-lon        lat  \
0         Capital Federal    3430787.0  -34.6618237,-58.5088387 -34.661824   
1  Bs.As. G.B.A. Zona Sur    3432039.0  -34.9038831,-57.9643295 -34.903883   
2         Capital Federal    3430787.0  -34.6522615,-58.5229825 -34.652262   

   ...  surface_covered_in_m2  price_usd_per_m2 price_per_m2  floor  rooms  \
0  ...                   40.0       1127.272727  1550.000000    NaN    NaN   
1  ...                    NaN               NaN          NaN    NaN    NaN   
2  ...                   55.0       1309.090909  1309.090909    NaN    NaN   

   expenses                                      properati_url  \
0       NaN  http://www.properati.com.ar/15bo8_venta_ph_mat...   
1       NaN  http://www.properati.com.ar/15bob_venta_depart...   
2       NaN  http://www.properati.com.ar/15bod_venta_depart...   

                                         description  \
0  2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...   
1  Venta de departamento en décimo piso al frente...   
2  2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...   

                                               title  \
0                 2 AMB TIPO CASA SIN EXPENSAS EN PB   
1  VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37   ...   
2           2 AMB 3ER PISO CON ASCENSOR APTO CREDITO   

                                     image_thumbnail  
0  https://thumbs4.properati.com/8/BluUYiHJLhgIIK...  
1  https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...  
2  https://thumbs4.properati.com/5/SXKr34F_IwG3W_...  

[3 rows x 26 columns]

In [5]:
# Eliminamos la columna Unnamed dado que son los mismos indices
data = data.drop(['Unnamed: 0'], axis=1)
data.head(3)

operation property_type place_name  \
0      sell            PH  Mataderos   
1      sell     apartment   La Plata   
2      sell     apartment  Mataderos   

                       place_with_parent_names country_name  \
0        |Argentina|Capital Federal|Mataderos|    Argentina   
1  |Argentina|Bs.As. G.B.A. Zona Sur|La Plata|    Argentina   
2        |Argentina|Capital Federal|Mataderos|    Argentina   

               state_name  geonames_id                  lat-lon        lat  \
0         Capital Federal    3430787.0  -34.6618237,-58.5088387 -34.661824   
1  Bs.As. G.B.A. Zona Sur    3432039.0  -34.9038831,-57.9643295 -34.903883   
2         Capital Federal    3430787.0  -34.6522615,-58.5229825 -34.652262   

         lon  ...  surface_covered_in_m2 price_usd_per_m2  price_per_m2  \
0 -58.508839  ...                   40.0      1127.272727   1550.000000   
1 -57.964330  ...                    NaN              NaN           NaN   
2 -58.522982  ...                   55.0      1309.090909   1309.090909   

   floor  rooms  expenses                                      properati_url  \
0    NaN    NaN       NaN  http://www.properati.com.ar/15bo8_venta_ph_mat...   
1    NaN    NaN       NaN  http://www.properati.com.ar/15bob_venta_depart...   
2    NaN    NaN       NaN  http://www.properati.com.ar/15bod_venta_depart...   

                                         description  \
0  2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...   
1  Venta de departamento en décimo piso al frente...   
2  2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...   

                                               title  \
0                 2 AMB TIPO CASA SIN EXPENSAS EN PB   
1  VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37   ...   
2           2 AMB 3ER PISO CON ASCENSOR APTO CREDITO   

                                     image_thumbnail  
0  https://thumbs4.properati.com/8/BluUYiHJLhgIIK...  
1  https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...  
2  https://thumbs4.properati.com/5/SXKr34F_IwG3W_...  

[3 rows x 25 columns]

# Limpieza de datos

## Analizamos que state_name vamos a utilizar

In [6]:
# Vemos todos los diferentes state_name que hay en el DataSet
data.state_name.unique()

array(['Capital Federal', 'Bs.As. G.B.A. Zona Sur',
       'Buenos Aires Costa Atlántica', 'Entre Ríos',
       'Bs.As. G.B.A. Zona Norte', 'Santa Fe', 'Córdoba',
       'Bs.As. G.B.A. Zona Oeste', 'Misiones', 'Buenos Aires Interior',
       'Salta', 'Neuquén', 'Río Negro', 'San Luis', 'Mendoza',
       'Corrientes', 'Chubut', 'Tucumán', 'La Pampa', 'Chaco', 'San Juan',
       'Santa Cruz', 'Tierra Del Fuego', 'Catamarca',
       'Santiago Del Estero', 'Jujuy', 'La Rioja', 'Formosa'],
      dtype=object)

In [7]:
# Vemos la cantidad de instancias que tiene cada state_name
state_name_distribution = pd.value_counts(data.state_name)
state_name_distribution

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
Santiago Del

In [8]:
# Vemos los porcentajes de la districion de los datos por provincia
total_data = data.shape[0]
percentage_state_name = 100*state_name_distribution/total_data
percentage_state_name

Capital Federal                 26.658967
Bs.As. G.B.A. Zona Norte        21.085629
Bs.As. G.B.A. Zona Sur          11.509652
Córdoba                          9.956278
Santa Fe                         8.391355
Buenos Aires Costa Atlántica     8.254413
Bs.As. G.B.A. Zona Oeste         7.690150
Buenos Aires Interior            1.889952
Río Negro                        0.666557
Neuquén                          0.604686
Mendoza                          0.561788
Tucumán                          0.556014
Corrientes                       0.480944
Misiones                         0.382775
Entre Ríos                       0.304405
Salta                            0.229335
Chubut                           0.213661
San Luis                         0.207886
La Pampa                         0.129517
Formosa                          0.053622
Chaco                            0.047022
San Juan                         0.032998
Tierra Del Fuego                 0.025573
Catamarca                        0

In [9]:
# Tomamos la decision de quedarnos con los datos que son > 1%, que corresponden a Buenos Aires Interior paara arriba
mask_1_percent = percentage_state_name[percentage_state_name > 1]
print(f'La muestra representa el {round(mask_1_percent.sum(), 2)}% del total de los datos')
print(round(mask_1_percent, 2))

La muestra representa el 95.44% del total de los datos
Capital Federal                 26.66
Bs.As. G.B.A. Zona Norte        21.09
Bs.As. G.B.A. Zona Sur          11.51
Córdoba                          9.96
Santa Fe                         8.39
Buenos Aires Costa Atlántica     8.25
Bs.As. G.B.A. Zona Oeste         7.69
Buenos Aires Interior            1.89
Name: state_name, dtype: float64


In [10]:
# Comprobamos la cantidad de filas donde tengan valor nulo en todas las columnas y las eliminamos  
data_clean_empty_row = data.dropna(axis = 0, how= 'all')
print(data.shape)
print(data_clean_empty_row.shape)
#En este caso no hay ninguna fila con todos los valores nulos

(121220, 25)
(121220, 25)


In [11]:
# Con la mascara generada anteriormente para quedarnos con los state_names mayores a 1%, eliminamos las instancias que no corresponden a los state_names seleccionados
data = data[data.state_name.isin(mask_1_percent.index)]
data

operation property_type       place_name  \
0           sell            PH        Mataderos   
1           sell     apartment         La Plata   
2           sell     apartment        Mataderos   
3           sell            PH          Liniers   
4           sell     apartment           Centro   
...          ...           ...              ...   
121215      sell     apartment         Belgrano   
121216      sell         house           Beccar   
121217      sell     apartment    Villa Urquiza   
121218      sell     apartment      Plaza Colón   
121219      sell     apartment  Capital Federal   

                                  place_with_parent_names country_name  \
0                   |Argentina|Capital Federal|Mataderos|    Argentina   
1             |Argentina|Bs.As. G.B.A. Zona Sur|La Plata|    Argentina   
2                   |Argentina|Capital Federal|Mataderos|    Argentina   
3                     |Argentina|Capital Federal|Liniers|    Argentina   
4       |Argentina|Buenos Aires Costa Atlántica|Mar de...    Argentina   
...                                                   ...          ...   
121215               |Argentina|Capital Federal|Belgrano|    Argentina   
121216  |Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...    Argentina   
121217          |Argentina|Capital Federal|Villa Urquiza|    Argentina   
121218  |Argentina|Buenos Aires Costa Atlántica|Mar de...    Argentina   
121219                        |Argentina|Capital Federal|    Argentina   

                          state_name  geonames_id  \
0                    Capital Federal    3430787.0   
1             Bs.As. G.B.A. Zona Sur    3432039.0   
2                    Capital Federal    3430787.0   
3                    Capital Federal    3431333.0   
4       Buenos Aires Costa Atlántica    3435548.0   
...                              ...          ...   
121215               Capital Federal    3436077.0   
121216      Bs.As. G.B.A. Zona Norte    3436080.0   
121217               Capital Federal    3433775.0   
121218  Buenos Aires Costa Atlántica          NaN   
121219               Capital Federal    3433955.0   

                              lat-lon        lat        lon  ...  \
0             -34.6618237,-58.5088387 -34.661824 -58.508839  ...   
1             -34.9038831,-57.9643295 -34.903883 -57.964330  ...   
2             -34.6522615,-58.5229825 -34.652262 -58.522982  ...   
3             -34.6477969,-58.5164244 -34.647797 -58.516424  ...   
4             -38.0026256,-57.5494468 -38.002626 -57.549447  ...   
...                               ...        ...        ...  ...   
121215                            NaN        NaN        NaN  ...   
121216                            NaN        NaN        NaN  ...   
121217  -34.5706388726,-58.4755963355 -34.570639 -58.475596  ...   
121218                            NaN        NaN        NaN  ...   
121219                            NaN        NaN        NaN  ...   

        surface_covered_in_m2 price_usd_per_m2  price_per_m2  floor  rooms  \
0                        40.0      1127.272727   1550.000000    NaN    NaN   
1                         NaN              NaN           NaN    NaN    NaN   
2                        55.0      1309.090909   1309.090909    NaN    NaN   
3                         NaN              NaN           NaN    NaN    NaN   
4                        35.0      1828.571429   1828.571429    NaN    NaN   
...                       ...              ...           ...    ...    ...   
121215                   93.0      7699.115044   9354.838710    NaN    NaN   
121216                  360.0      1383.333333   1383.333333    NaN    NaN   
121217                   39.0      2858.695652   3371.794872    NaN    NaN   
121218                   48.0      1997.916667   1997.916667    NaN    NaN   
121219                   77.0      1675.324675   1675.324675    NaN    NaN   

        expenses                                      properati_url  \
0            NaN  http://www.properati.com.

## Analis preliminar del DataSet

In [12]:
# Usamos dtypes para saber que tipo de datos se almacenan en el dataset
data.dtypes

operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: object

In [13]:
#Vemos cuantos valores nulos tenemos por columna
is_null_result = data.isnull().sum()
print(is_null_result)

operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18527
lat-lon                        48639
lat                            48639
lon                            48639
price                          17870
currency                       17871
price_aprox_local_currency     17870
price_aprox_usd                17870
surface_total_in_m2            37058
surface_covered_in_m2          17744
price_usd_per_m2               49058
price_per_m2                   30175
floor                         108073
rooms                          69989
expenses                      101758
properati_url                      0
description                        2
title                              0
image_thumbnail                 2848
dtype: int64


In [14]:
data_cero = (data==0).sum()
data_cero

operation                       0
property_type                   0
place_name                      0
place_with_parent_names         0
country_name                    0
state_name                      0
geonames_id                     0
lat-lon                         0
lat                             0
lon                             0
price                           1
currency                        0
price_aprox_local_currency      1
price_aprox_usd                 1
surface_total_in_m2           379
surface_covered_in_m2           2
price_usd_per_m2                0
price_per_m2                    0
floor                           0
rooms                           0
expenses                        0
properati_url                   0
description                     0
title                           0
image_thumbnail                 0
dtype: int64

In [15]:
#Vemos que porcentaje de nulos tiene cada columna
percentage_null = 100*is_null_result/data.shape[0]
percentage_null

operation                      0.000000
property_type                  0.000000
place_name                     0.019881
place_with_parent_names        0.000000
country_name                   0.000000
state_name                     0.000000
geonames_id                   16.014626
lat-lon                       42.043254
lat                           42.043254
lon                           42.043254
price                         15.446719
currency                      15.447583
price_aprox_local_currency    15.446719
price_aprox_usd               15.446719
surface_total_in_m2           32.032709
surface_covered_in_m2         15.337805
price_usd_per_m2              42.405435
price_per_m2                  26.083086
floor                         93.417641
rooms                         60.498064
expenses                      87.958993
properati_url                  0.000000
description                    0.001729
title                          0.000000
image_thumbnail                2.461794


## Tratamiento de nulos en la columna place_name

In [16]:
#Analizamos los nulos en la columna place_name con una mascara booleana
place_name = data.place_name
mask_place_name = place_name.isnull()
data_null_place_name = data[mask_place_name]
data_null_place_name

operation property_type place_name  \
6489       sell         house        NaN   
10201      sell         house        NaN   
11451      sell         house        NaN   
14839      sell         house        NaN   
18622      sell         house        NaN   
21922      sell         house        NaN   
23664      sell         house        NaN   
24722      sell         house        NaN   
38856      sell         house        NaN   
45970      sell     apartment        NaN   
46642      sell         house        NaN   
53130      sell         house        NaN   
55306      sell         house        NaN   
57703      sell         house        NaN   
57759      sell         house        NaN   
57764      sell         house        NaN   
57793      sell         house        NaN   
58004      sell         house        NaN   
58037      sell         house        NaN   
59069      sell         house        NaN   
62411      sell         house        NaN   
62413      sell         house        NaN   
63849      sell         house        NaN   

                           place_with_parent_names country_name  \
6489   |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
10201  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
11451  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
14839  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
18622  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
21922  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
23664  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
24722  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
38856  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
45970  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
46642  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
53130  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
55306  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
57703  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
57759  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
57764  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
57793  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
58004  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
58037  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
59069  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
62411  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
62413  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   
63849  |Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    Argentina   

                     state_name  geonames_id                        lat-lon  \
6489   Bs.As. G.B.A. Zona Norte          NaN    -34.4008968545,-58.63809847   
10201  Bs.As. G.B.A. Zona Norte          NaN          -34.400704,-58.638067   
11451  Bs.As. G.B.A. Zona Norte          NaN  -34.4026880384,-58.6262613379   
14839  Bs.As. G.B.A. Zona Norte          NaN  -34.4007994779,-58.6381735719   
18622  Bs.As. G.B.A. Zona Norte          NaN  -34.4032781195,-58.6263503945   
21922  Bs.As. G.B.A. Zona Norte          NaN          -34.399704,-58.638405   
23664  Bs.As. G.B.A. Zona Norte          NaN  -34.4005161998,-58.6417999185   
24722  Bs.As. G.B.A. Zona Norte          NaN        -34.4007292,-58.6377783   
38856  Bs.As. G.B.A. Zona Norte          NaN         -34.425087,-58.5796585   
45970  Bs.As. G.B.A. Zona Norte          NaN           -34.396074,-58.63738   
46642  Bs.As. G.B.A. Zona Norte          NaN          -34.400749,-58.638754   
53130  Bs.As. G.B.A. Zona Norte          NaN          -34.400978,-58.637809   
55306  Bs.As. G.B.A. Zona Norte          NaN        -34.3996311,-58.6433121   
57703  Bs.As. G.B.A. Zona Norte          NaN      -34.39033749,-58.63368988   
57759  Bs.As. G.B.A. Zona Norte          NaN      -34.39596815,-58.63077164   
57764  Bs.As. G.B.A. Zona Norte          NaN      -34.39635767,-58.62613678 

In [17]:
# Como todos los nulos en la columna place_name corresponden al municipio de Tigre, los imputamos directamente con este municipio
data.loc[data_null_place_name.index, "place_name"] = 'Tigre'

## Analisis de columnas surface_total_in_m2 y surface_covered_in_m2

In [18]:
# Vemos en que casos el valor surface_total_in_m2 es menor que el valor surface_covered_in_m2
mask_surface = data[data.surface_total_in_m2 < data.surface_covered_in_m2]
mask_surface[["surface_total_in_m2" , "surface_covered_in_m2"]]

surface_total_in_m2  surface_covered_in_m2
177                   420.0                  450.0
316                   125.0                  156.0
317                   125.0                  156.0
455                   170.0                  430.0
668                   180.0                  230.0
...                     ...                    ...
120168                120.0                  140.0
120181                153.0                  226.0
120202                 70.0                  200.0
120302               1200.0                12000.0
121055                 77.0                   98.0

[1070 rows x 2 columns]

In [19]:
#En los casos en que surface_total_in_m2 sea menor que surface_covered_in_m2 imputamos los datos en el orden inverso

list_a = mask_surface.surface_total_in_m2
list_b = mask_surface.surface_covered_in_m2

data.loc[mask_surface.index, "surface_total_in_m2"] = list_b
data.loc[mask_surface.index, "surface_covered_in_m2"] = list_a

In [25]:
# Realizamos una mascara para ver las instancias de superficie tanto cubierta como total de los cuales no tenemos para imputar un valor con el otro

mask_surface_total_in_m2_zero = data[((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 == 0) | (data.surface_covered_in_m2.isnull()))]

print (mask_surface_total_in_m2_zero[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])

#En este DataFrame hay tanto superficies nulas como superficies iguales a cero

       property_type  surface_total_in_m2  surface_covered_in_m2
1          apartment                  NaN                    NaN
3                 PH                  NaN                    NaN
9              house                  NaN                    NaN
11             house                  NaN                    NaN
65             store                  NaN                    NaN
...              ...                  ...                    ...
121070     apartment                  NaN                    NaN
121079     apartment                  NaN                    NaN
121101         house                  NaN                    NaN
121134         house                  NaN                    NaN
121144         house                  NaN                    NaN

[11437 rows x 3 columns]


In [28]:
# Realizamos una mascara para ver que valores tenemos nulos o ceros de superficie total pero que la cubierta no es nula ni cero, por lo tanto se la podemos imputar

mask_surface_total_in_m2_zero = data[((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 != 0) & (data.surface_covered_in_m2.notnull()))]

print (mask_surface_total_in_m2_zero[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])

#Como el DataFrame no es nulo, vamos a poder imputar los valores nulos o ceros de superficie total con la superficie cubierta

       property_type  surface_total_in_m2  surface_covered_in_m2
68         apartment                  NaN                   72.0
77         apartment                  NaN                   85.0
79         apartment                  NaN                   42.0
87             house                  NaN                  170.0
89         apartment                  NaN                   53.0
...              ...                  ...                    ...
121155         house                  NaN                  130.0
121156         house                  NaN                  130.0
121207         house                  NaN                  178.0
121208         house                  NaN                  178.0
121210         house                  NaN                  120.0

[26000 rows x 3 columns]


In [29]:
# Realizamos una mascara para ver que valores tenemos nulos o ceros de superficie cubierta pero que la total no es nula ni cero, por lo tanto se la podemos imputar

mask_surface_covered_in_m2_zero = data[((data.surface_total_in_m2 != 0) | (data.surface_total_in_m2.notnull())) & ((data.surface_covered_in_m2 == 0) & (data.surface_covered_in_m2.isnull()))]

print (mask_surface_covered_in_m2_zero[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])

#Como el DataFrame es nulo, no vamos a poder imputar los valores nulos o ceros de superficie cubierta con la total

Empty DataFrame
Columns: [property_type, surface_total_in_m2, surface_covered_in_m2]
Index: []


In [31]:
# Aca imputamos los valores nulos o ceros de superficie total con los valores de superficie cubierta, utilizando la mascara mask_surface_total_in_m2_zero

data.loc[mask_surface_total_in_m2_zero.index, "surface_total_in_m2"] = data.loc[mask_surface_total_in_m2_zero.index, "surface_covered_in_m2"]

#Aca imputamos las 

## Analisis de columna rooms

### En las instancias que rooms sea igual a cero o nulo, imputamos el valor por medio de una expresion regular aplicada a la columna de descripcion. 

In [ ]:
# 1. Mascara booleana que contenga cero o nulo de rooms
boolean_mask_applied = data[(data.rooms == 0) | (data.rooms.isnull())]
boolean_mask_applied

In [ ]:
# 2. Usamos expresiones regulares para sacar los rooms en descripcion
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras amb
patron = '(?P<rooms>\d+)\s+(?i)amb'  
pattern_nombre_regex = re.compile(patron)

In [ ]:
# Serie de la columna "despcription"
description_series = boolean_mask_applied.description
description_series

In [ ]:
# Mostramos donde hay matches
resultado_rooms = description_series.apply(lambda x: pattern_nombre_regex.search(str(x)))
print(resultado_rooms)

In [ ]:
# Extraemos los datos que usaremos para la imputación
rooms_match = resultado_rooms.apply(lambda x: x if x is None else x.group('rooms'))
rooms_match

In [ ]:
# 3. Remplazar el nuevo data series segun sus indices en la data. 

In [ ]:
# Queremos, para los que tengan ambas superficies nulas o cero se las imputamos por una media condicionada por el valor de property_type, place_name, state_name y rooms

In [ ]:
# Vemos cuantos unique hay en place_name para ver que no exitan dos place_names iguales para dos state_names diferentes (ej. centro)

data.place_name.unique()

In [ ]:
# Vemos los que tienen rooms nulos y superficies nulas
mas_rooms = (data.rooms.isnull() & data.surface_covered_in_m2.isnull() & data.surface_total_in_m2.isnull()) or (data.rooms == 0 & data.surface_covered_in_m2 == 0 & data.surface_total_in_m2.isnull == 0 & data.surface_total_in_m2 == 0)
data_rooms = data[mask_rooms]
data_rooms[["property_type", "place_name" , "state_name" ,"rooms", "surface_covered_in_m2" , "surface_total_in_m2" ]]